In [215]:
import simpy
import numpy as np
import statistics

process_times = []


In [216]:
class Hospital(object):
    def __init__(self, env, num_nurses, num_radiologists, num_specialists):
        self.env = env
        self.nurse = simpy.Resource(env, num_nurses)
        self.radiologist = simpy.Resource(env, num_radiologists)
        self.specialist = simpy.Resource(env, num_specialists)

    def check_vitals(self, patient):
        yield self.env.timeout(np.random.randint(5, 10))
    
    def take_xray(self, patient):
        yield self.env.timeout(np.random.randint(15, 25))
    
    def consult_patient(self, patient):
        yield self.env.timeout(np.random.randint(10, 20))


In [217]:
def go_to_hospital(env, patient, hospital):
    # Patient arrives at the hospital
    arrival_time = env.now

    with hospital.nurse.request() as request:
        yield request
        print(f"Nurse starts checking vitals of patient {patient} at {env.now}")
        yield env.process(hospital.check_vitals(patient))
        print(f"Nurse finished checking patient {patient} at {env.now}")
    
    if np.random.choice([True, False]):
        with hospital.radiologist.request() as request:
            yield request
            print(f"Radiologist takes xray of patient {patient} at {env.now}")
            yield env.process(hospital.take_xray(patient))
            print(f"Radiologist releases patient {patient} at {env.now}")

    with hospital.specialist.request() as request:
        yield request
        print(f"Specialist starts consult with patient {patient} at {env.now}")
        yield env.process(hospital.consult_patient(patient))
        print(f"Specialist finished consult with patient {patient} at {env.now}")

    # Patient goes home
    finish_time = env.now
    process_time = finish_time - arrival_time
    print(f">> Patient {patient} started at {arrival_time} and left at {finish_time} with total process time {process_time}")
    process_times.append(process_time)


In [218]:
def run_hospital(env, num_nurses, num_radiologists, num_specialists):
    hospital = Hospital(env, num_nurses, num_radiologists, num_specialists)
    
    for patient in range(3):
        env.process(go_to_hospital(env, patient, hospital))

    while True:
        yield env.timeout(0.20)  # Wait a bit before generating a new patient

        patient += 1
        env.process(go_to_hospital(env, patient, hospital))


In [219]:
def assess_process_quality(process_times):
    average_process = statistics.mean(process_times)
    num_patients = len(process_times)
    return average_process, num_patients


In [220]:
def main(params=[1, 1, 1]):
    # Setup
    np.random.seed(42)
    num_nurses, num_radiologists, num_specialists = params

    # Run the simulation
    env = simpy.Environment()
    env.process(run_hospital(env, num_nurses, num_radiologists, num_specialists))
    print("Running simulation...")
    env.run(until=90)

    # View the results
    average_process, num_patients = assess_process_quality(process_times)
    print(f"\nThe average process time is {average_process} minutes with {num_patients} patients handled.",)


In [222]:
params=[1,1,1]
main(params)


Running simulation...
Nurse starts checking vitals of patient 0 at 0
Nurse finished checking patient 0 at 8
Radiologist takes xray of patient 0 at 8
Nurse starts checking vitals of patient 1 at 8
Nurse finished checking patient 1 at 17
Nurse starts checking vitals of patient 2 at 17
Nurse finished checking patient 2 at 23
Nurse starts checking vitals of patient 3 at 23
Radiologist releases patient 0 at 30
Nurse finished checking patient 3 at 30
Specialist starts consult with patient 0 at 30
Radiologist takes xray of patient 1 at 30
Nurse starts checking vitals of patient 4 at 30
Nurse finished checking patient 4 at 38
Nurse starts checking vitals of patient 5 at 38
Nurse finished checking patient 5 at 45
Nurse starts checking vitals of patient 6 at 45
Specialist finished consult with patient 0 at 47
>> Patient 0 started at 0 and left at 47 with total process time 47
Specialist starts consult with patient 4 at 47
Radiologist releases patient 1 at 49
Radiologist takes xray of patient 2 a